In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
#run_time = '28-01-2025_18:28:13'
#run_time = '28-01-2025_18:33:30'

run_time = '09-02-2025_01:19:00'
#run_time = '09-02-2025_03:05:57'
#run_time = '10-02-2025_00:20:55'
#run_time = '10-02-2025_02:16:16'



## Drawing confidence band

In [4]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Open and read the JSON file
path = '../out/run_' + run_time + '/'
with open(path + 'run_info.json', 'r') as file:
    run_info = json.load(file)

# Create a single figure
plt.figure(figsize=(10, 6))

# Iterate over algorithms and plot confidence intervals
for algorithm in run_info['algorithms']:
    algorithm_undersored = algorithm.replace(' ', '_')

    points = pd.read_csv(path + algorithm_undersored + '_points.csv')
    bound = pd.read_csv(path + algorithm_undersored + '_bound.csv')

    # Calculate mean and standard deviation of points for each k
    k_values = points['k'].unique()
    k_values = np.sort(k_values)
    mean_values = np.array([np.sqrt(points[points['k'] == k]['value']).mean() for k in k_values])
    std_values = np.array([np.sqrt(points[points['k'] == k]['value']).std() for k in k_values])
    upper_values = mean_values + 1.96 * std_values
    lower_values = mean_values - 1.96 * std_values

    # Plot confidence interval
    plt.plot(k_values, mean_values, label=algorithm, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), dashes=[5, 5])
    plt.plot(k_values, upper_values, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), alpha = 0.0)
    plt.plot(k_values, lower_values, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), alpha = 0.0)
    plt.fill_between(k_values, lower_values, upper_values, alpha=0.33, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)))

    # Check if bound is all zeros
    if not (bound['value'] == 0).all():
        plt.plot(bound['k'], np.sqrt(bound['value']), label=algorithm + ' bound', linestyle=':', color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)))

# Set title and labels
plt.title('Confidence Intervals for all algorithms')
plt.xlabel('k')
plt.ylabel(r'$\Vert X \Vert_2 / \Vert X_S \Vert_2$')

# Show legend
plt.legend()

# Save plot
plt.tight_layout()
plt.savefig(path + 'confidence_intervals_plot.svg')
plt.close()

In [11]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# LaTeX configuration and style setup
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 12,
    "axes.titlesize": 12,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "grid.color": "0.8",
    "grid.linestyle": "--",
    "grid.linewidth": 0.6,
    "lines.linewidth": 1.8,
    "axes.linewidth": 0.8,
    "patch.edgecolor": "black"
})

# Document dimensions
textwidth_pt = 345  # Typical value for 12pt LaTeX doc with 2cm margins
inches_per_pt = 1/72.27
half_textwidth = textwidth_pt * inches_per_pt * 0.5

# Plot parameters
FIG_SIZE = (half_textwidth, half_textwidth * 0.618)  # Golden ratio
SHOW_LEGEND = True  #Set this to True when needed
SHOW_TITLE = False  # Typically better to use LaTeX captions

def create_plot(run_time):
    path = '../out/run_' + run_time + '/'
    with open(path + 'run_info.json', 'r') as file:
        run_info = json.load(file)

    fig, ax = plt.subplots(figsize=FIG_SIZE)

    for idx, algorithm in enumerate(run_info['algorithms']):
        algorithm_underscored = algorithm.replace(' ', '_')
        
        points = pd.read_csv(path + algorithm_underscored + '_points.csv')
        bound = pd.read_csv(path + algorithm_underscored + '_bound.csv')

        k_values = np.sort(points['k'].unique())
        sqrt_values = np.sqrt(points['value'])
        
        mean_values = sqrt_values.groupby(points['k']).mean()
        std_values = sqrt_values.groupby(points['k']).std()
        ci = 1.96 * std_values

        color = plt.cm.tab10(idx)
        
        # Main plot with confidence interval
        ax.plot(k_values, mean_values, color=color, 
                dashes=[6, 2])
        ax.fill_between(k_values, mean_values - ci, mean_values + ci,
                        color=color, alpha=0.2, linewidth=0)

        # Theoretical bound
        if not (bound['value'] == 0).all():
            ax.plot(bound['k'], np.sqrt(bound['value']), 
                    color=color, linestyle=':', linewidth=1.2)

    # Axis labels
    ax.set_xlabel(r'$k$', labelpad=8)
    ax.set_ylabel(r'$\Vert X \Vert_2 / \Vert X_S \Vert_2$', labelpad=10)
    
    if SHOW_TITLE:
        ax.set_title('Algorithm Comparison', pad=15)

    # Grid and borders
    ax.set_axisbelow(True)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Legend configuration

    # Create custom legend
    if SHOW_LEGEND:
        custom_legend = []
        for algorithm in run_info['algorithms']:
            color = plt.cm.tab10(list(run_info['algorithms']).index(algorithm))
            custom_legend.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=algorithm, markersize=10))
        
        leg = ax.legend(handles=custom_legend, framealpha=0.9, 
                       loc='upper left', handlelength=2.6, 
                       handletextpad=0.5)
        leg.get_frame().set_linewidth(0.8)
        leg.get_frame().set_edgecolor("0.8")

    plt.tight_layout(pad=0.8)
    plt.savefig(path + 'confidence_intervals_plot.pdf', bbox_inches='tight')
    plt.close()

# Example usage
create_plot(run_time)

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpb66p713y d174ba5a1278c3ea3324ab2ddb79f45c.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./d174ba5a1278c3ea3324ab2ddb79f45c.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages of 

Error in callback <function _draw_all_if_interactive at 0x7b21551bcea0> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp3hnw1i03 d174ba5a1278c3ea3324ab2ddb79f45c.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./d174ba5a1278c3ea3324ab2ddb79f45c.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages of 

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp8y3tu79x d174ba5a1278c3ea3324ab2ddb79f45c.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./d174ba5a1278c3ea3324ab2ddb79f45c.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages of 

<Figure size 238.688x147.509 with 1 Axes>

## Russian version

In [53]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.size'] = 16

# Open and read the JSON file
path = '../out/run_' + run_time + '/'
with open(path + 'run_info.json', 'r') as file:
    run_info = json.load(file)

# Create a single figure
plt.figure(figsize=(10, 6))
alg_names = ["новый алгоритм",
             "алгоритм 2 из [1]",
      "алгоритм 3 из [1]",
      "случайные столбцы"]

# Iterate over algorithms and plot confidence intervals
i = 0
for algorithm in run_info['algorithms']:
    algorithm_undersored = algorithm.replace(' ', '_')

    points = pd.read_csv(path + algorithm_undersored + '_points.csv')
    bound = pd.read_csv(path + algorithm_undersored + '_bound.csv')

    # Calculate mean and standard deviation of points for each k
    k_values = points['k'].unique()
    k_values = np.sort(k_values)
    mean_values = np.array([np.sqrt(points[points['k'] == k]['value']).mean() for k in k_values])
    std_values = np.array([np.sqrt(points[points['k'] == k]['value']).std() for k in k_values])
    upper_values = mean_values + 1.96 * std_values
    lower_values = mean_values - 1.96 * std_values

    # Plot confidence interval
    plt.plot(k_values, mean_values)
    plt.plot(k_values, upper_values, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), alpha = 0.0)
    plt.plot(k_values, lower_values, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)), alpha = 0.0)
    plt.fill_between(k_values, lower_values, upper_values, alpha=0.33, color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)))

    # Check if bound is all zeros
    if not (bound['value'] == 0).all():
        plt.plot(bound['k'], np.sqrt(bound['value']), linestyle='--', color=plt.cm.tab10(list(run_info['algorithms']).index(algorithm)))
    i += 1

# Set title and labels
plt.xlabel('k')
plt.ylabel(r'$\Vert X^\dag \Vert_2 / \Vert X_S^\dag \Vert_2$')
# Create custom legend
custom_legend = []
i = 0
for algorithm in run_info['algorithms']:
    color = plt.cm.tab10(list(run_info['algorithms']).index(algorithm))
    custom_legend.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=alg_names[i], markersize=10))
    i+=1
plt.legend(handles=custom_legend, loc='best')

# Save plot
plt.tight_layout()
plt.savefig(path + 'confidence_intervals_plot_ru.svg')
plt.close()